## assistant 만들어보기

### 0-1. api_key 불러오기

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 0-2. Assistant 응답 결과를 json 형태로 반환하는 함수 만들기

In [3]:
import json

def response_to_dict(obj):
    return json.loads(obj.model_dump_json())

### 1-1. 생성된 assistant 확인

In [4]:
from openai import OpenAI
client = OpenAI()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
for i in my_assistants.data:
    print(i)

Assistant(id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', created_at=1712988472, description=None, file_ids=['file-DqdIDulmB8mP3GCjAk3NGTEX', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-rTNSIUTUjG5l1yikWILW9pa1'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if the question is too broad or if it cannot find the exact information in the document. It aims to eng

#### 1-2. assistant 만들기
- 현재 있는 어시스턴트 확인하기
- 생성된 어시스턴트가 없을 경우 이름으로 생성하기
- 생성된 어시스턴트의 이름으로 id 조회하기
- id 조회로 어시스턴트 

In [23]:
assistant_names = []
for i in my_assistants.data:
    assistant_names.append(i.name)
print("현재 있는 어시스턴트 종류는 :", assistant_names)

현재 있는 어시스턴트 종류는 : []


In [24]:
# 생성된 어시스턴트가 없을 경우 생성
my_assistant_name = "데이브 더 다이브 게임 가이드"
if my_assistant_name not in assistant_names:
    my_assistant = client.beta.assistants.create(
    instructions = "This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if the question is too broad or if it cannot find the exact information in the document. It aims to engage users by providing insightful, game-related information and facilitating a deeper understanding of the game's universe. Respond in Korean only",
    name = my_assistant_name,
    tools = [{"type": "retrieval"}],
    model = "gpt-4-turbo",
)

In [25]:
for i in my_assistants.data:
    if i.name == my_assistant_name:
        my_assistant_id = i.id
print(my_assistant_id)

asst_XckrgwZWlubKaYC3wagUMnVB


### 1-3. 검색용 파일 업로드

In [26]:
### txt 파일 리스트
from glob import glob
txt_list = glob("./*.txt")
txt_list

['.\\등장인물.txt',
 '.\\무기.txt',
 '.\\부재료 및 조미료.txt',
 '.\\식당 운영.txt',
 '.\\요리.txt',
 '.\\장비.txt',
 '.\\지역 및 생물.txt']

In [27]:
# 파일 객체 생성
file_id_list = []
for file_path in txt_list:
  upload_file_path = file_path
  file = client.files.create(
    file=open(upload_file_path, "rb"),
    purpose='assistants'
  )
  file_id_list.append(file.id)

In [28]:
file_id_list

['file-rTNSIUTUjG5l1yikWILW9pa1',
 'file-qPfDpph7Ouhi4Y51K4sObBgP',
 'file-CoVYbf5ZNpYnYEML2FoPMdqn',
 'file-lgVC1S54zKZOUkeGLtuJZX4A',
 'file-4GMJdPNpAJvR8vJ1zv0sgjBI',
 'file-HyeZozp2SL4fjaR2EZDPvVZq',
 'file-DqdIDulmB8mP3GCjAk3NGTEX']

### 1-4. 어시스턴트 업데이트

In [29]:
# 기존의 어시스턴트에 파일 업로드 하기 - instructions, model  수정가능
client.beta.assistants.update(
    assistant_id=my_assistant.id,
    file_ids=file_id_list,
)

Assistant(id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', created_at=1712988472, description=None, file_ids=['file-rTNSIUTUjG5l1yikWILW9pa1', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-DqdIDulmB8mP3GCjAk3NGTEX'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if the question is too broad or if it cannot find the exact information in the document. It aims to eng

In [30]:
## 파일 검색해보기
assistant_file = client.beta.assistants.files.retrieve(
  assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ", # 생성된 assistant_id 를 입력해주세요
  file_id="file-rTNSIUTUjG5l1yikWILW9pa1" # 찾고자 하는 file id를 입력해주세요
)
print(assistant_file)

AssistantFile(id='file-rTNSIUTUjG5l1yikWILW9pa1', assistant_id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', created_at=1712988518, object='assistant.file')


### 1-5. 파일 삭제

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id="어시스턴트 id",
  file_id="삭제할 파일 id"
)

### 2-1. 대화 스레드 생성

In [84]:
empty_thread = client.beta.threads.create() # 스레드 생성 : 새 대화창이라고 생각
print(empty_thread)

Thread(id='thread_OZNg0Ieeau9R51bxN8of3xPN', created_at=1712993021, metadata={}, object='thread')


In [85]:
my_thread = client.beta.threads.retrieve(empty_thread.id) # 대화창 검색해보기
print(my_thread)

Thread(id='thread_OZNg0Ieeau9R51bxN8of3xPN', created_at=1712993021, metadata={}, object='thread')


### 3-1. 메시지 객체 생성

In [86]:
thread_message = client.beta.threads.messages.create(
    thread_id=my_thread.id,
    role="user",
    content="데이브 더 다이브의 등장인물에 대해서 요약해줘",
)
print(thread_message)

Message(id='msg_tW2U3nkl9yPOzWqjCG5uNLfe', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='데이브 더 다이브의 등장인물에 대해서 요약해줘'), type='text')], created_at=1712993029, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_OZNg0Ieeau9R51bxN8of3xPN')


In [87]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=my_thread.id,
  assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ" # 현재 하고 있는 어시스턴트 id
)
print(run)

Run(id='run_IzltKmOYjh86pd953NgLHPv3', assistant_id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', cancelled_at=None, completed_at=1712993095, created_at=1712993068, expires_at=None, failed_at=None, file_ids=['file-rTNSIUTUjG5l1yikWILW9pa1', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-DqdIDulmB8mP3GCjAk3NGTEX'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if th

### 4-1. 생성된 내용 출력해보기 - 동기 방식

In [88]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=my_thread.id
  )
  response_data = response_to_dict(messages)
  for msg_data in response_data["data"]:
      print(msg_data["content"][0]["text"])
else:
  print(run.status)

{'annotations': [{'end_index': 755, 'file_citation': {'file_id': 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'quote': ''}, 'start_index': 744, 'text': '【17†source】', 'type': 'file_citation'}], 'value': '게임 "데이브 더 다이버"에는 다양한 등장인물들이 있으며 이들은 크게 몇 개의 그룹으로 구분할 수 있습니다.\n\n1. **반쵸스시**:\n   - **데이브**: 주인공으로, 배가 나온 다이버 아저씨입니다.\n   - **코브라**: 데이브를 블루홀로 초대하는 인물입니다.\n   - **반쵸**: 이름이 그대로 인물의 이름입니다.\n   - **모모**: 등장인물 중 하나입니다.\n   - **모찌**: 등장인물 중 하나입니다.\n\n2. **주변 인물**:\n   - **엘리**, **더프**, **닥터 베이컨**, **요시에**, **사토상**, **우도**, **오토**, **마키** 등 다양한 인물들이 있습니다. 각각 게임의 스토리와 상황에서 중요한 역할을 합니다.\n\n3. **VIP**:\n   - **빈센트**, **마이클 방**, **새미** 등 높은 지위를 가지고 있거나 중요한 역할을 하는 인물들입니다.\n   - **챌린지 더 쉐프** 섹션에는 **왕팡**, **알렉스**, **파스트로** 등이 속해 있습니다.\n\n4. **어인족 마을**:\n   - **라모**, **텐진**, **수왐**, **가진** 등이 속한, 어인족으로 구성된 마을의 주요 인물들입니다.\n   - 다양한 어인족 인물들이 등장하며 게임 내에서 독특한 역할을 담당하고 있습니다.\n\n5. **기타 인물**:\n   - **해적**, **존 왓슨**, **루이스**, **유이**, **클라라** 등 게임 내에서 다양한 배경과 이야기를 갖고 있는 인물들입니다【17†source】.'}
{'annotations': [], 'value':

### 4-2. 메시지 추가해보기

In [89]:
thread_message = client.beta.threads.messages.create(
    thread_id=my_thread.id,
    role="user",
    content="데이브 더 다이브의 무기에 대해서 요약해줘",
)
print(thread_message)

Message(id='msg_FIaTedo3PH58LLHRU5K8STbl', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='데이브 더 다이브의 무기에 대해서 요약해줘'), type='text')], created_at=1712993192, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_OZNg0Ieeau9R51bxN8of3xPN')


In [90]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=my_thread.id,
  assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ"
)
print(run)

Run(id='run_uW6xPr3UKkOV1XjO7q5KLIH7', assistant_id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', cancelled_at=None, completed_at=1712993221, created_at=1712993206, expires_at=None, failed_at=None, file_ids=['file-rTNSIUTUjG5l1yikWILW9pa1', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-DqdIDulmB8mP3GCjAk3NGTEX'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if th

In [91]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=my_thread.id
  )
  response_data = response_to_dict(messages)
  for msg_data in response_data["data"]:
      print(msg_data["content"][0]["text"])
else:
  print(run.status)

{'annotations': [{'end_index': 127, 'file_citation': {'file_id': 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'quote': ''}, 'start_index': 114, 'text': '【24:0†source】', 'type': 'file_citation'}, {'end_index': 252, 'file_citation': {'file_id': 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'quote': ''}, 'start_index': 239, 'text': '【24:0†source】', 'type': 'file_citation'}, {'end_index': 372, 'file_citation': {'file_id': 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'quote': ''}, 'start_index': 359, 'text': '【24:0†source】', 'type': 'file_citation'}], 'value': '게임 "데이브 더 다이버"에서의 무기에 대한 설명은 다음과 같습니다:\n\n1. **무기 상자**: 무기 상자에서는 무기를 랜덤하게 얻을 수 있습니다. 무기 상자를 열었을 때 나오는 무기는 자동으로 사용됩니다【24:0†source】.\n\n2. **장거리 수중 스쿠터**: 대용량 배터리가 탑재된 스쿠터로, 속도는 느려졌지만 사용 시간이 개선되어 오랜요 보조 장비로 활용됩니다. 이동 속도 50% 증가 및 사용 시간 20초를 제공합니다【24:0†source】.\n\n3. **설치형 트리거 폭탄**: 강력한 폭발을 일으킬 수 있는 설치형 트리거 폭탄입니다. 터뜨리면 곧장 대상에게 큰 피해를 입힐 수 있으며, 폭발 피해는 40, 폭발 범위는 3M 입니다【24:0†source】.\n\n이 게임에서 무기는 대부분 잠수하기 전에 쉽게 구매하거나 발견할 수 있으며, 게임 내 특정 퀘스트나 도전에서 중요한 역할을 합니다.'}
{'annotations':

### 5-1.파일 지정해서 검색해보기
- 등장인물 : 'file-rTNSIUTUjG5l1yikWILW9pa1'
- 무기 :'file-qPfDpph7Ouhi4Y51K4sObBgP'
- 부재료 및 조미료 : 'file-CoVYbf5ZNpYnYEML2FoPMdqn'
- 식당 운영 : 'file-lgVC1S54zKZOUkeGLtuJZX4A'
- 요리 : 'file-4GMJdPNpAJvR8vJ1zv0sgjBI'
- 장비 : 'file-HyeZozp2SL4fjaR2EZDPvVZq'
- 지역 및 생물 : 'file-DqdIDulmB8mP3GCjAk3NGTEX'

In [8]:
my_thread2 = client.beta.threads.create() # 스레드 생성 : 새 대화창이라고 생각
thread_message2 = client.beta.threads.messages.create(
    thread_id=my_thread2.id,
    role="user",
    content="데이브 더 다이브의 등장인물에 대해서 요약해줘",
    file_ids = ["file-rTNSIUTUjG5l1yikWILW9pa1"]
)
run = client.beta.threads.runs.create_and_poll(
  thread_id=my_thread2.id,
  assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ" # 현재 하고 있는 어시스턴트 id,
)
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=my_thread2.id
  )
  response_data = response_to_dict(messages)
  for msg_data in response_data["data"]:
      print(msg_data["content"][0]["text"]["value"])
else:
  print(run.status)

**데이브 더 다이버**에서 등장하는 주요 캐릭터와 그룹에 대한 요약은 다음과 같습니다:

1. **데이브**: 게임의 주인공으로 다이버이자 아저씨. 어느 열대의 섬에서 휴양 중에 코브라의 초밥집 제안을 받고 블루홀에 오게 됩니다. 초밥 재료를 직접 잡아오면서 가게 운영도 하게 됩니다. 매일매일 바닷속에서 잠수하고 밤에는 서빙까지 하는 등, 게임을 통해 초인적인 체력을 보여줍니다.

2. **코브라**: 데이브와 반쵸를 블루홀의 생선을 이용한 초밥집 차림으로 모은 인물. 메인 스토리 클리어 후에는 다이빙 장비를 판매하고 다이빙에서 얻은 물건들을 매입하는 코브라샵을 운영합니다.

3. **반쵸**: 게임 중 흑인 초밥 장인으로, 미지의 영역을 탐사하는 것을 목표로 합니다. 평범한 식용 생선 대신 열대어나 해파리, 상어, 심해어 같은 미지의 재료만을 다루어 높은 요리 수준을 보여줍니다.

4. **모모와 모찌**: 부가적인 정보는 없으며, 이름만 언급됨.

5. **주변 인물**: 엘리, 더프, 닥터 베이컨, 요시에, 사토상 등 다양한 인물들이 나타나며 각각 특별한 역할을 수행합니다.

6. **어인족 마을**: 다양한 인물로 구성되어 있으며, 데이브는 이 마을을 구하고 마을 사람들은 데이브를 기리기 위해 석상을 세웁니다.

이 게임은 캐릭터들이 각기 독특한 역할이 있으며, 흥미로운 이야기꾼들로 구성되어 있습니다. Detailed descriptions of other characters and other aspects can be provided if interested.
데이브 더 다이브의 등장인물에 대해서 요약해줘
